In [12]:
import os
import numpy as np
import pandas as pd

# === AJUSTA ESTAS RUTAS A TU ESTRUCTURA ===
DATA_DIR = "data"  # carpeta de salida para el app
os.makedirs(DATA_DIR, exist_ok=True)

# Insumos que ya tienes en el notebook (ajusta si tu nombre de archivo es otro)
PATH_MOVIES      = "../../data/dataset/final/X_trained_scaled_final_no_cat.csv"                         # contiene movieId y title (y si tienes year, genres, poster_path, mejor)
PATH_CLUSTERS    = "train_clusters_kmeans.csv"          # tiene una columna 'cluster' (mismo orden que tu X)
PATH_FEATURE_CSV = "../../data/dataset/final/X_trained_scaled_final_no_cat.csv"  # tu matriz de features (con o sin movieId)

movies   = pd.read_csv(PATH_MOVIES)         # columnas esperadas: movieId, title, (opcional year, genres, poster_path)
clusters = pd.read_csv(PATH_CLUSTERS)       # columna 'cluster' (N filas)
X_df     = pd.read_csv(PATH_FEATURE_CSV)    # si trae movieId en la primera col, abajo lo manejamos


In [13]:
# Detectar si la primera columna parece ser movieId
first_col = X_df.columns[0]
has_movie_id_in_X = (first_col.lower() in ["movieid", "movie_id", "id"])

if has_movie_id_in_X:
    X_movie_ids = X_df[first_col].astype(int).to_numpy()
    X_values = X_df.drop(columns=[first_col]).to_numpy(dtype=float)
else:
    X_movie_ids = None
    X_values = X_df.to_numpy(dtype=float)

N, D = X_values.shape
print(f"Features: N={N}, D={D}")


Features: N=5233, D=1981


In [14]:
# Normalizamos columnas mínimas
movies_cols = movies.columns.str.lower().tolist()
if "movieid" not in movies_cols:
    raise ValueError("Tu 'movies.csv' debe tener columna 'movieId'.")

# Columnas opcionales
if "title" not in movies.columns:
    movies["title"] = ""

if "year" not in movies.columns:
    movies["year"] = np.nan

if "genres" not in movies.columns:
    movies["genres"] = ""

if "poster_path" not in movies.columns:
    # Puedes dejar vacío o construir rutas/URLs reales si las tienes
    movies["poster_path"] = ""   # ejemplo: "posters/{movieId}.jpg"


In [15]:
# Validar longitud de clusters
if "cluster" not in clusters.columns:
    raise ValueError("train_clusters_kmeans.csv debe tener columna 'cluster'.")
clusters_arr = clusters["cluster"].to_numpy()
if len(clusters_arr) != len(X_values):
    raise ValueError(f"clusters ({len(clusters_arr)}) y X ({len(X_values)}) no tienen la misma longitud.")

# Alineación:
# - Si X tiene movieId: ordenamos movies según X_movie_ids.
# - Si X NO tiene movieId: asumimos que el orden de movies coincide con X y clusters.

if X_movie_ids is not None:
    # Validar que todos los movieId de X existan en movies
    missing = set(X_movie_ids) - set(movies["movieId"].astype(int))
    if missing:
        raise ValueError(f"Hay movieId en X que no están en movies: {list(missing)[:10]} ...")

    # Reordenar movies al orden de X_movie_ids
    movies_indexed = movies.set_index("movieId").loc[X_movie_ids].reset_index()
else:
    # Asegurar que movies tenga misma longitud y orden que X/clusters
    if len(movies) != len(X_values):
        raise ValueError("X no tiene movieId y su longitud no coincide con movies. Necesitas alinear explícitamente.")
    movies_indexed = movies.copy()

# Exportar features.npy
np.save(os.path.join(DATA_DIR, "features.npy"), X_values)

# Exportar cluster_labels.csv (una sola columna 'cluster')
pd.DataFrame({"cluster": clusters_arr}).to_csv(
    os.path.join(DATA_DIR, "cluster_labels.csv"), index=False
)

# Exportar metadata.csv con columnas requeridas
metadata = movies_indexed[["movieId", "title", "year", "genres", "poster_path"]].copy()
metadata.to_csv(os.path.join(DATA_DIR, "metadata.csv"), index=False)

print("✅ Exportados:")
print(" - data/features.npy")
print(" - data/cluster_labels.csv")
print(" - data/metadata.csv")


✅ Exportados:
 - data/features.npy
 - data/cluster_labels.csv
 - data/metadata.csv


In [16]:
# Sanity check de tamaños
features = np.load(os.path.join(DATA_DIR, "features.npy"))
clusters_check = pd.read_csv(os.path.join(DATA_DIR, "cluster_labels.csv"))["cluster"].to_numpy()
meta_check = pd.read_csv(os.path.join(DATA_DIR, "metadata.csv"))

assert features.shape[0] == len(clusters_check) == len(meta_check), "Tamaños no coinciden."
print("Todo OK: ", features.shape, len(clusters_check), len(meta_check))

# Opcional: revisar n_clusters y ejemplos
print("n_clusters =", len(np.unique(clusters_check)))
display(meta_check.head())


Todo OK:  (5233, 1981) 5233 5233
n_clusters = 5


,movieId,title,year,genres,poster_path
0,619,NaN,NaN,NaN,NaN
1,33826,NaN,NaN,NaN,NaN
2,1298,NaN,NaN,NaN,NaN
3,140289,NaN,NaN,NaN,NaN
4,3064,NaN,NaN,NaN,NaN
